# Supply Points Analysis between Grab and Sensors

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import random

import plotly.graph_objects as go

import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

import umap

In [ ]:
seed = 42

In [ ]:
utils_folder = os.path.join("..", "..", "utils")

data_folder = os.path.join("..", "..", "data")
clean_data_folder = os.path.join(data_folder, "Clean Data")
metadata_folder = os.path.join(data_folder, "Metadata")
plot_folder = os.path.join(data_folder, "Plots")

sensor_folder = os.path.join(clean_data_folder, "sensors")

nuwee_folder = os.path.join(plot_folder, "nuwee")

# Load Data

In [ ]:
grab_df = pd.read_excel(os.path.join(clean_data_folder, "grab.xlsx"))

In [ ]:
sensor_dict = {}

for file in os.listdir(sensor_folder):
    if file.endswith(".xlsx"):
        sensor_dict[file.split(".")[0]] = pd.read_excel(
            os.path.join(sensor_folder, file)
        )

In [ ]:
with open(os.path.join(utils_folder, "columns_types.json")) as f:
    column_types = json.load(f)

metadata_columns = column_types["metadata_columns"]
features_columns = column_types["features_columns"]
targets_columns = column_types["targets_columns"]

In [ ]:
grab_df

In [ ]:
from operator import contains


label_columns = [col for col in grab_df.columns if contains(col, "label")]

In [ ]:
label_columns

In [ ]:
# rename grab columns
feature_mapping = {
    "Cloro residuo libero (al prelievo) (mg/L di Cl2)": "Free Chlorine (mg/L)",
    "Colore (Cu)": "Color (CU)",
    "Concentrazione ioni idrogeno (unità pH)": "pH",
    "Conduttività a 20°C (µS/cm)": "Conductivity (uS/cm)",
    "TOC - carbonio organico totale (mg/L di C)": "TOC (mg/L)",
    "Temperatura (al prelievo) (°C)": "Temperature (°C)",
    # "Torbidità (NTu)": "Turbidity (NTU)",
    "Nitrati (mg/L)": "Nitrate (mg/L)",
}

targets_mapping = {
    "Bromodiclorometano (µg/L)": "Dichlorobromomethane (ug/L)",
    "Bromoformio (µg/L)": "Bromoform (ug/L)",
    "Cloroformio (µg/L)": "Chloroform (ug/L)",
    "Dibromoclorometano (ug/L)": "Dibromochloromethane (ug/L)",
}

In [ ]:
# rename grab_df columns that contain features
for column in grab_df.columns:
    if column in targets_mapping:
        grab_df.rename(columns={column: targets_mapping[column]}, inplace=True)

    if len(column.split("_")) > 1:
        if column.split("_")[0] in feature_mapping:
            new_name = feature_mapping[column.split("_")[0]]
            new_name = new_name + "_" + column.split("_")[1]
            grab_df.rename(columns={column: new_name}, inplace=True)

        if column.split("_")[0] in targets_mapping:
            new_name = targets_mapping[column.split("_")[0]]
            new_name = new_name + "_" + column.split("_")[1]
            grab_df.rename(columns={column: new_name}, inplace=True)

In [ ]:
# rename the label columns
for column in grab_df.columns:
    if column in label_columns:
        variable_name = column.split("_")[0]

        if variable_name in feature_mapping:
            new_name = feature_mapping[variable_name]
            new_name = new_name + "_" + column.split("_")[1]
            grab_df.rename(columns={column: new_name}, inplace=True)

        if variable_name in targets_mapping:
            new_name = targets_mapping[variable_name]
            new_name = new_name + "_" + column.split("_")[1]
            grab_df.rename(columns={column: new_name}, inplace=True)

In [ ]:
grab_df

# Metadata Info

In [ ]:
grab_df = grab_df[grab_df['DateTime'] > '2024-01-01']

In [ ]:
grab_df

## Grab

In [ ]:
feature_df = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
        [
            feature_mapping.values(),
            [
                "N° Entries",
                "N° Valid Samples",
                "% Missing",
                "N° < LOQ",
            ],
        ]
    ),
    index=grab_df["Code"].unique(),
)

In [ ]:
for code in grab_df["Code"].unique():
    for feature in feature_mapping.values():
        df = grab_df[grab_df["Code"] == code][
            ["DateTime", feature, feature + "_label"]
        ].copy()

        if df.dropna().shape[0] == 0:
            continue

        df["DateTime"] = pd.to_datetime(df["DateTime"])

        start_date = df.dropna()["DateTime"].min().strftime("%Y-%m-%d")
        end_date = df.dropna()["DateTime"].max().strftime("%Y-%m-%d")

        df = df[(df["DateTime"] >= start_date) & (df["DateTime"] <= end_date)]

        missing_values = (
            df[df[feature + "_label"].isna()].shape[0] / df.shape[0] * 100
        )

        feature_df.loc[code, (feature, "N° Entries")] = df.shape[0]

        feature_df.loc[code, (feature, "% Missing")] = round(missing_values, 2)

        feature_df.loc[code, (feature, "N° < LOQ")] = df[
            df[feature + "_label"] == "Less than"
        ].shape[0]
        
        
        valid_df = df[df[feature + "_label"] == "Normal"]
        loq_df = df[df[feature + "_label"] == "Less than"]
        
        feature_df.loc[code, (feature, "N° Valid Samples")] = valid_df.shape[0]
        feature_df.loc[code, (feature, "N° < LOQ")] = loq_df.shape[0]

In [ ]:
feature_df

In [ ]:
# sort the indexes
feature_df.sort_index(inplace=True)

In [ ]:
# sort the first level of the columns and maintain the order of the second level
feature_df = feature_df.sort_index(axis=1, level=0, sort_remaining=False, key=lambda x: x.str.lower())

In [ ]:
targets_df = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
        [
            targets_mapping.values(),
            [
                "N° Entries",
                "N° Valid Samples",
                "% Missing",
                "N° < LOQ",
            ],
        ]
    ),
    index=grab_df["Code"].unique(),
)

In [ ]:
for code in grab_df["Code"].unique():
    for target in targets_mapping.values():
        df = grab_df[grab_df["Code"] == code][
            ["DateTime", target, target + "_label"]
        ].copy()

        if df.dropna().shape[0] == 0:
            continue

        df["DateTime"] = pd.to_datetime(df["DateTime"])

        start_date = df.dropna()["DateTime"].min().strftime("%Y-%m-%d")
        end_date = df.dropna()["DateTime"].max().strftime("%Y-%m-%d")

        df = df[(df["DateTime"] >= start_date) & (df["DateTime"] <= end_date)]

        missing_values = (
            df[df[target + "_label"].isna()].shape[0] / df.shape[0] * 100
        )

        targets_df.loc[code, (target, "N° Entries")] = df.shape[0]

        valid_df = df[df[target + "_label"] == "Normal"]
        loq_df = df[df[target + "_label"] == "Less than"]
        
        
        targets_df.loc[code, (target, "% Missing")] = round(missing_values, 2)

        targets_df.loc[code, (target, "N° Valid Samples")] = valid_df.shape[0]
        targets_df.loc[code, (target, "N° < LOQ")] = loq_df.shape[0]

In [ ]:
targets_df

In [ ]:
targets_df.sort_index(inplace=True)

In [ ]:
targets_df = targets_df.sort_index(axis=1, level=0, sort_remaining=False, key=lambda x: x.str.lower())

In [ ]:
# %%script false --no-raise-error
feature_df.to_excel(os.path.join(metadata_folder, "Grab", "features.xlsx"))

targets_df.to_excel(os.path.join(metadata_folder, "Grab", "targets.xlsx"))

## Sensor

In [ ]:
### Fix Conductivity name
for sensor in sensor_dict:
    sensor_dict[sensor].rename(
        columns={"Conductivity (μS/cm)": "Conductivity (uS/cm)"}, inplace=True
    )

In [ ]:
sensor_columns = sensor_dict["Berna"].columns.difference(["DateTime"])

In [ ]:
sensor_columns

In [ ]:
sensors_df = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
        [sensor_columns, ["N° Data", "N° Missing", "Mean", "Std", "Start Date", "End Date"]]
    ),
    index=list(sensor_dict.keys()),
)

In [ ]:
for sensor in sensor_dict.keys():
    for column in sensor_columns:
        if sensor == "Berna" and column == "Turbidity (FTU)":
            df = sensor_dict[sensor].copy()
            # remove rows with Turbidity > 2
            df = df[df["Turbidity (FTU)"] <= 2]

            sensors_df.loc[sensor, (column, "N° Data")] = df[column].count()
            sensors_df.loc[sensor, (column, "N° Missing")] = (
                df[column].isna().sum()
            )
            sensors_df.loc[sensor, (column, "Mean")] = df[column].mean()
            sensors_df.loc[sensor, (column, "Std")] = df[column].std()
            continue

        sensors_df.loc[sensor, (column, "N° Data")] = sensor_dict[sensor][
            column
        ].count()
        sensors_df.loc[sensor, (column, "N° Missing")] = (
            sensor_dict[sensor][column].isna().sum()
        )
        sensors_df.loc[sensor, (column, "Mean")] = sensor_dict[sensor][
            column
        ].mean()
        sensors_df.loc[sensor, (column, "Std")] = sensor_dict[sensor][
            column
        ].std()
        
        start_date = sensor_dict[sensor]["DateTime"].min().strftime("%Y-%m-%d")
        end_date = sensor_dict[sensor]["DateTime"].max().strftime("%Y-%m-%d")
        
        sensors_df.loc[sensor, (column, "Start Date")] = start_date
        sensors_df.loc[sensor, (column, "End Date")] = end_date

In [ ]:
sensors_df.sort_index(inplace=True)

In [ ]:
sensors_df

In [ ]:
sensors_df.to_excel(os.path.join(metadata_folder, "Sensor", "sensors.xlsx"))

# Scatter Plot Pair

In [ ]:
# scatter plot pair grid for grab features
fig = plt.figure(figsize=(20, 20))

sns.pairplot(grab_df, vars=feature_mapping.values(), hue="Code")

plt.show()

# Time Series Filtering

In [ ]:
%%script false --no-raise-error

n_hours = 3

for code in sensor_dict:
    sensor_df = sensor_dict[code].copy()
    
    for column in sensor_df.columns:
        if column == 'DateTime':
            continue
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle(f'{code} - {column}', fontsize=16)
        
        # Before Moving Average
        sns.lineplot(ax=axes[0, 0], x=sensor_df['DateTime'], y=sensor_df[column], color='blue')
        axes[0, 0].set_title('Before MA')
        axes[0, 0].set_xlabel('Time')
        axes[0, 0].set_ylabel(column)
        axes[0, 0].grid()
        
        # After Moving Average
        df = sensor_df[['DateTime', column]].copy()
        df.set_index('DateTime', inplace=True)
        
        df = df[~df.index.duplicated(keep='first')]
        
        df = df.rolling(window=4*n_hours).mean()
        sns.lineplot(ax=axes[0, 1], x=sensor_df['DateTime'], y=sensor_df[column], color='blue', alpha=0.3)
        sns.lineplot(ax=axes[0, 1], x=df.index, y=df[column], color='green')
        axes[0, 1].set_title('After MA')
        axes[0, 1].set_xlabel('Time')
        axes[0, 1].set_ylabel(column)
        axes[0, 1].grid()
        
        # Histogram Before Moving Average
        sns.histplot(ax=axes[1, 0], data=sensor_df, x=column, color='purple', kde=True, stat='density')
        axes[1, 0].set_title('Before MA')
        axes[1, 0].set_xlabel(column)
        axes[1, 0].set_ylabel('Density')
        
        # Histogram After Moving Average
        sns.histplot(ax=axes[1, 1], data=sensor_df, x=column, color='purple', stat='density', alpha=0.3)
        sns.histplot(ax=axes[1, 1], data=df, x=column, color='green', kde=True, stat='density')
        axes[1, 1].set_title('After MA')
        axes[1, 1].set_xlabel(column)
        axes[1, 1].set_ylabel('Density')
    
    
        # add a legend of the number of hours
        fig.legend(['Before MA', 'After MA'], loc='upper right')
        
        
        plt.tight_layout()
        
        path = os.path.join(plot_folder, 'Clean Data', 'Moving Average', code)
        
        if not os.path.exists(path):
            os.makedirs(path)
            
        column_ = column.replace('/', '_')
        
        fig.savefig(os.path.join(path, column_ + '.png'), dpi=300)
        plt.close(fig)
        
        # plt.show()


# Time Series Comparison

In [ ]:
# %%script false --no-raise-error

# plot the time series of the sensors and the grab data


n_hours = 3

for code in grab_df["Code"].unique():
    for feature in feature_mapping.values():
        g_df = grab_df[grab_df["Code"] == code].copy()
        
        g_df = g_df[["DateTime", feature]].copy()
        g_df.dropna(inplace=True)

        s_df = sensor_dict[code].copy()
        
        start_date = s_df["DateTime"].min().strftime("%Y-%m-%d")
        
        g_df = g_df[g_df["DateTime"] >= start_date]

        # moving average on sensor data

        ma_s_df = s_df.copy()

        ma_s_df.set_index("DateTime", inplace=True)
        ma_s_df = ma_s_df.rolling(window=4 * n_hours).mean()

        
        std = g_df[feature].std()

        fig = go.Figure()

        fig.add_trace(
            go.Scatter(
                x=s_df["DateTime"], y=s_df[feature], mode="lines", name="Sensor"
            )
        )

        # fig.add_trace(
        #     go.Scatter(
        #         x=ma_s_df.index,
        #         y=ma_s_df[feature],
        #         mode="lines",
        #         name="Sensor MA",
        #         line=dict(color="green"),
        #     )
        # )

        fig.add_trace(
            go.Scatter(
                x=g_df["DateTime"],
                y=g_df[feature],
                mode="markers",
                name="Grab",
                marker=dict(size=12, color="red"), 
            )
        )
        
        # add the std to each point of the grab data

        fig.update_layout(
            title=f"{code} - {feature}",
            xaxis_title="DateTime",
            yaxis_title=feature,
            # put legend inside the plot
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1,
            ),
            margin=dict(l=0, r=0, t=0, b=0),
            hovermode="closest",

        )

        if not os.path.exists(
            os.path.join(plot_folder, "Comparison", "15min", "Timeseries", code)
        ):
            os.makedirs(
                os.path.join(
                    plot_folder, "Comparison", "15min", "Timeseries", code
                )
            )

        feature_ = feature.replace("/", "_")

        fig.write_image(
            os.path.join(
                plot_folder,
                "Comparison",
                "15min",
                "Timeseries",
                code,
                f"{feature_}.png",
            ),
            height=600,
            width=1200,
            scale=3,
        )

        # fig.show()

In [ ]:
# thms_columns = ['Bromodichloromethane (µg/L)', 'Bromoform (µg/L)', 'Chloroform (µg/L)', 'Dibromochloromethane (µg/L)']

In [ ]:
# grab_df['TTHMs'] = grab_df[thms_columns].sum(axis=1, min_count=len(thms_columns))

In [ ]:
# %%script false --no-raise-error

# plot all the grab samples points

n_hours = 3

for feature in feature_mapping.values():
    
    fig = go.Figure()
    
    for code in grab_df["Code"].unique():
        g_df = grab_df[grab_df["Code"] == code].copy()
        
        g_df = g_df[["DateTime", feature]].copy()
        g_df.dropna(inplace=True)

        s_df = sensor_dict[code].copy()
        
        start_date = s_df["DateTime"].min().strftime("%Y-%m-%d")
        
        g_df = g_df[g_df["DateTime"] >= start_date]

        # moving average on sensor data

        fig.add_trace(
            go.Scatter(
                x=g_df["DateTime"],
                y=g_df[feature],
                mode="markers+lines",
                name=f"{code}",
                marker=dict(size=12),
            )
        )
        
        # add the std to each point of the grab data

    fig.update_layout(
        title=f"{feature}",
        xaxis_title="DateTime",
        yaxis_title=feature,
        # put legend inside the plot
        legend=dict(
            x=1,
            y=1,
            traceorder="normal",
        ),
        margin=dict(l=0, r=0, t=30, b=0),
        hovermode="closest",

    )

    feature_ = feature.replace("/", "_")

    # fig.write_image(
    #     f"{feature_}.png",
    #     scale=3,
    # )
    
    fig.show()

# Boxplot Comparison 

In [ ]:
def month_to_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    if month in [3, 4, 5]:
        return "Spring"
    if month in [6, 7, 8]:
        return "Summer"
    if month in [9, 10, 11]:
        return "Autumn"

In [ ]:
%%script --false --no-raise-error

# plot the box plot of grab data and sensor data
for code in grab_df["Code"].unique():
    for feature in feature_mapping.values():
        g_df = grab_df[grab_df["Code"] == code].copy()

        s_df = sensor_dict[code].copy()

        # moving average on sensor data
        s_df["DateTime"] = pd.to_datetime(s_df["DateTime"])
        s_df.set_index("DateTime", inplace=True)
        s_df = s_df.rolling(window=4 * n_hours).mean()

        sensor_start_date = s_df.index.min().strftime("%Y-%m-%d")

        g_df["DateTime"] = pd.to_datetime(g_df["DateTime"])
        
        before_g_df = g_df[g_df["DateTime"] < sensor_start_date]
        after_g_df = g_df[g_df["DateTime"] >= sensor_start_date]
        
        valid_g_df = g_df[g_df[feature + "_label"] == "Normal"]
        loq_g_df = g_df[g_df[feature + "_label"] == "Less than"]
        
        valid_before_g_df = valid_g_df[valid_g_df["DateTime"] < sensor_start_date]
        valid_after_g_df = valid_g_df[valid_g_df["DateTime"] >= sensor_start_date]
        
        loq_before_g_df = loq_g_df[loq_g_df["DateTime"] < sensor_start_date]
        loq_after_g_df = loq_g_df[loq_g_df["DateTime"] >= sensor_start_date]

        # divide before and after into seasons
        valid_before_g_df["Season"] = valid_before_g_df["DateTime"].dt.month.apply(
            month_to_season
        )
        valid_after_g_df["Season"] = valid_after_g_df["DateTime"].dt.month.apply(
            month_to_season
        )

        loq_before_g_df["Season"] = loq_before_g_df["DateTime"].dt.month.apply(
            month_to_season
        )
        loq_after_g_df["Season"] = loq_after_g_df["DateTime"].dt.month.apply(
            month_to_season
        )

        fig = make_subplots(
            rows=3,
            cols=1,
            specs=[[{"type": "xy"}], [{"type": "table"}], [{"type": "table"}]],
            subplot_titles=(
                "",
                f"Grab Samples Before {sensor_start_date}",
                f"Grab Samples After {sensor_start_date}",
            ),
        )

        fig.add_trace(
            go.Box(
                y=valid_before_g_df[feature],
                name=f"Valid Old Grab<br>N° Points: {valid_before_g_df[feature].count()}",
            ),
            row=1,
            col=1,
        )
        
        fig.add_trace(
            go.Box(
                y=loq_before_g_df[feature],
                name=f"LOQ Old Grab<br>N° Points: {loq_before_g_df[feature].count()}",
            ),
            row=1,
            col=1,
        )
        
        fig.add_trace(
            go.Box(
                y=before_g_df[feature],
                name=f"Overall Old Grab<br>N° Points: {before_g_df[feature].count()}",
            ),
            row=1,
            col=1,
        )
        
        fig.add_trace(
            go.Box(
                y=valid_after_g_df[feature],
                name=f"Valid New Grab<br>N° Points: {valid_after_g_df[feature].count()}",
            ),
            row=1,
            col=1,
        )
        
        fig.add_trace(
            go.Box(
                y=loq_after_g_df[feature],
                name=f"LOQ New Grab<br>N° Points: {loq_after_g_df[feature].count()}",
            ),
            row=1,
            col=1
        )
        
        fig.add_trace(
            go.Box(
                y=after_g_df[feature],
                name=f"Overall New Grab<br>N° Points: {after_g_df[feature].count()}",
            ),
            row=1,
            col=1
        )
        

        fig.add_trace(go.Box(y=s_df[feature], name="Sensor"), row=1, col=1)

        # divide by season for both old and new grab data
        fig.add_trace(
            go.Table(
                header=dict(
                    values=["Season Valid", "N° Points Valid", "Mean Valid", "Std Valid", "Season LOQ", "N° Points LOQ", "Mean LOQ", "Std LOQ"],
                    align="center",
                ),
                cells=dict(
                    values=[
                        valid_before_g_df.groupby("Season").size().index,
                        valid_before_g_df.groupby("Season")[feature].count().values,
                        valid_before_g_df.groupby("Season")[feature]
                        .mean()
                        .values.round(2),
                        valid_before_g_df.groupby("Season")[feature]
                        .std()
                        .values.round(2),
                        loq_before_g_df.groupby("Season").size().index,
                        loq_before_g_df.groupby("Season")[feature].count().values,
                        loq_before_g_df.groupby("Season")[feature]
                        .mean()
                        .values.round(2),
                        loq_before_g_df.groupby("Season")[feature]
                        .std()
                        .values.round(2),
                    ],
                    align="center",
                ),
            ),
            row=2,
            col=1,
        )

        fig.add_trace(
            go.Table(
                header=dict(
                    values=["Season Valid", "N° Points Valid", "Mean Valid", "Std Valid", "Season LOQ", "N° Points LOQ", "Mean LOQ", "Std LOQ"],
                    align="center",
                ),
                cells=dict(
                    values=[
                        valid_after_g_df.groupby("Season").size().index,
                        valid_after_g_df.groupby("Season")[feature].count(),
                        valid_after_g_df.groupby("Season")[feature]
                        .mean()
                        .values.round(2),
                        valid_after_g_df.groupby("Season")[feature]
                        .std()
                        .values.round(2),
                        loq_after_g_df.groupby("Season").size().index,
                        loq_after_g_df.groupby("Season")[feature].count(),
                        loq_after_g_df.groupby("Season")[feature]
                        .mean()
                        .values.round(2),
                        loq_after_g_df.groupby("Season")[feature]
                        .std()
                        .values.round(2),
                    ],
                    align="center",
                ),
            ),
            row=3,
            col=1,
        )

        fig.update_layout(
            title=f"{code} - {feature}",
            yaxis_title=feature,
        )
        
        fig.add_annotation(
            dict(
                x=-0.022,
                y=1.07,
                xref="paper",
                yref="paper",
                showarrow=False,
                text=f"Grab Samples divided by date {sensor_start_date}",
                font=dict(size=12, color="gray"),
            )
        )

        if not os.path.exists(
            os.path.join(plot_folder, "Comparison", "Daily", "Boxplot", code)
        ):
            os.makedirs(
                os.path.join(
                    plot_folder, "Comparison", "Daily", "Boxplot", code
                )
            )

        feature_ = feature.replace("/", "_")

        fig.write_image(
            os.path.join(
                plot_folder, "Comparison", "Daily", "Boxplot", code, f'{feature_}.png',
            ),
            height=800,
            width=1200,
            scale=3
        )

        fig.show()

# Bland-Altman Test

In [ ]:
import seaborn as sns

In [ ]:
%%script false --no-raise-error
# With all the supply points together


total_g_df = pd.DataFrame(columns=["Code", "DateTime", "Feature", "Value"])
total_s_df = pd.DataFrame(columns=["Code", "DateTime", "Feature", "Value"])

for code in grab_df["Code"].unique():
    for feature in feature_mapping.values():
        g_df = grab_df[grab_df["Code"] == code].copy()
        
        # if code == "Berna" and feature == "Free Chlorine (mg/L)":
        #     pass

        s_df = sensor_dict[code].copy()

        # moving average on sensor data
        s_df["DateTime"] = pd.to_datetime(s_df["DateTime"])
        s_df.set_index("DateTime", inplace=True)
        # 2 hours moving average
        s_df = s_df.rolling(window=4 * 2).mean()
        
        # fix the date of the sensor data to have a frequency of 15 minutes for easier comparison and interpolate to not have nan value
        s_df = s_df.resample("15min").mean().interpolate(method="time")

        sensor_start_date = s_df.index.dropna().min().strftime("%Y-%m-%d")
        sensor_end_date = s_df.dropna().index.max().strftime("%Y-%m-%d")
        
        g_df.set_index("DateTime", inplace=True)
        
        g_df = g_df[(g_df.index >= sensor_start_date) & (g_df.index <= sensor_end_date)]
        
        g_df = g_df[feature]
        g_df.dropna(inplace=True)
        
        # keep only the sensor values that have the date in the grab data and the hour is between 9 and 11
        
        dates = pd.Series(s_df.index.date, index=s_df.index).isin(g_df.index.date)
        dates = dates[dates.values]
        
        s_df = s_df.loc[dates.index]
        s_df = s_df[(s_df.index.hour == 10) & (s_df.index.minute >= 0) & (s_df.index.minute <= 14)]
        
        # if there is more than one value for the same date, take the mean
        s_df = s_df.groupby(s_df.index.date).mean()
        
        total_g_df = pd.concat([total_g_df, pd.DataFrame({"Code": code, "DateTime": g_df.index, "Feature": feature, "Value": g_df.values})])
        total_s_df = pd.concat([total_s_df, pd.DataFrame({"Code": code, "DateTime": s_df.index, "Feature": feature, "Value": s_df[feature].values})])


for feature in feature_mapping.values():
    
    g_df = total_g_df[total_g_df["Feature"] == feature]
    s_df = total_s_df[total_s_df["Feature"] == feature]
    
    g_df["DateTime"] = pd.to_datetime(g_df["DateTime"])
    s_df["DateTime"] = pd.to_datetime(s_df["DateTime"])
    
        
    df = pd.merge(g_df, s_df, on=["Code", "DateTime"], suffixes=("_Grab", "_Sensor"))
    df["Difference"] = df["Value_Grab"] - df["Value_Sensor"]
    df['Mean'] = (df["Value_Grab"] + df["Value_Sensor"]) / 2
    

    difference_mean = np.mean(df["Difference"].values)
    difference_std = np.std(df["Difference"].values)
    std_error = difference_std / np.sqrt(g_df.shape[0])

    ci_difference_mean = 1.96 * std_error
    
    
    f, ax = plt.subplots(1, 1, figsize=(15, 10))
    sns.scatterplot(data=df, x="Mean", y="Difference", hue="Code", ax=ax, s=100)
    
    ax.axhline(y=difference_mean, color='green', linestyle='--', label='Mean')
    
    if feature == "Conductivity (uS/cm)":
        ax.text(x=712, y=difference_mean, s=f'Mean: {difference_mean:.2f}', color='green')
    
    else:
        ax.text(x=df['Mean'].quantile(0.9), y=difference_mean + std_error, s=f'Mean: {difference_mean:.2f}', color='green')
    
    ax.axhline(y=difference_mean + 1.96 * difference_std, color='red', linestyle='--', label='1.96 * Std')
    # add text over the horizontal line
    
    if feature == "Conductivity (uS/cm)":
        ax.text(x=712, y=difference_mean + 1.96 * difference_std , s=f'+ 1.96 * Std', color='red')
        
    else:
        ax.text(x=df['Mean'].quantile(0.9), y=difference_mean + 1.96 * difference_std + std_error, s=f'+ 1.96 * Std', color='red')
    
    ax.axhline(y=difference_mean - 1.96 * difference_std, color='red', linestyle='--', label='-1.96 * Std')
    # add text over the horizontal line
    
    if feature == "Conductivity (uS/cm)":
        ax.text(x=712, y=difference_mean - 1.96 * difference_std, s=f'-1.96 * Std', color='red')
        
    else:
        ax.text(x=df['Mean'].quantile(0.9), y=difference_mean - 1.96 * difference_std + std_error, s=f'-1.96 * Std', color='red')
    
    ax.axhline(y=0, color='black', linestyle='--')


    plt.annotate(
        f'Std error: {std_error:.2f}\nDifference mean CI: {difference_mean:.2f} ± {ci_difference_mean:.2f}',
        xy=(0.6, 0.94),
        xycoords='axes fraction',
        bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="lightblue"),
        color='green',
        fontsize=14
    )
    
    plt.title(f'{feature} - Bland-Altman Plot')
    plt.tight_layout(pad=2)
    
    # increment the font size
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(16)
        
    for item in ax.get_legend().get_texts():
        item.set_fontsize(14)

    # display Bland-Altman plot
    if not os.path.exists(os.path.join(plot_folder, "Bland-Altman", "Supply Points", "All")):
        os.makedirs(os.path.join(plot_folder, "Bland-Altman", "Supply Points", "All"))
        
    feature_ = feature.replace("/", "_")

    plt.savefig(os.path.join(plot_folder, "Bland-Altman", "Supply Points", "All", f'{feature_}.png'), dpi=300)
    # plt.close(f)

    plt.show()

In [ ]:
%%script false --no-raise-error

# With all the supply points together

total_g_df = pd.DataFrame(columns=["Code", "DateTime", "Feature", "Value"])
total_s_df = pd.DataFrame(columns=["Code", "DateTime", "Feature", "Value"])

for code in grab_df["Code"].unique():
    for feature in feature_mapping.values():
        g_df = grab_df[grab_df["Code"] == code].copy()
        
        # if code == "Berna" and feature == "Free Chlorine (mg/L)":
        #     pass

        s_df = sensor_dict[code].copy()

        # moving average on sensor data
        s_df["DateTime"] = pd.to_datetime(s_df["DateTime"])
        s_df.set_index("DateTime", inplace=True)
        # 2 hours moving average
        s_df = s_df.rolling(window=4 * 2).mean()
        
        # fix the date of the sensor data to have a frequency of 15 minutes for easier comparison and interpolate to not have nan value
        s_df = s_df.resample("15min").mean().interpolate(method="time")

        sensor_start_date = s_df.index.dropna().min().strftime("%Y-%m-%d")
        sensor_end_date = s_df.dropna().index.max().strftime("%Y-%m-%d")
        
        g_df.set_index("DateTime", inplace=True)
        
        g_df = g_df[(g_df.index >= sensor_start_date) & (g_df.index <= sensor_end_date)]
        
        g_df = g_df[feature]
        g_df.dropna(inplace=True)
        
        # keep only the sensor values that have the date in the grab data and the hour is between 9 and 11
        
        dates = pd.Series(s_df.index.date, index=s_df.index).isin(g_df.index.date)
        dates = dates[dates.values]
        
        s_df = s_df.loc[dates.index]
        s_df = s_df[(s_df.index.hour == 10) & (s_df.index.minute >= 0) & (s_df.index.minute <= 14)]
        
        # if there is more than one value for the same date, take the mean
        s_df = s_df.groupby(s_df.index.date).mean()
        
        total_g_df = pd.concat([total_g_df, pd.DataFrame({"Code": code, "DateTime": g_df.index, "Feature": feature, "Value": g_df.values})])
        total_s_df = pd.concat([total_s_df, pd.DataFrame({"Code": code, "DateTime": s_df.index, "Feature": feature, "Value": s_df[feature].values})])


for feature in feature_mapping.values():
    
    g_df = total_g_df[total_g_df["Feature"] == feature]
    s_df = total_s_df[total_s_df["Feature"] == feature]
    
    g_df["DateTime"] = pd.to_datetime(g_df["DateTime"])
    s_df["DateTime"] = pd.to_datetime(s_df["DateTime"])
    
        
    df = pd.merge(g_df, s_df, on=["Code", "DateTime"], suffixes=("_Grab", "_Sensor"))
    df["Difference"] = df["Value_Grab"] - df["Value_Sensor"]
    df['Mean'] = (df["Value_Grab"] + df["Value_Sensor"]) / 2
    

    difference_mean = np.mean(df["Difference"].values)
    difference_std = np.std(df["Difference"].values)
    std_error = difference_std / np.sqrt(g_df.shape[0])

    ci_difference_mean = 1.96 * std_error
    
    
    f, ax = plt.subplots(1, 1, figsize=(15, 10))
    sns.scatterplot(data=df, x="DateTime", y="Difference", hue="Code", ax=ax, s=100)
    
    ax.axhline(y=difference_mean, color='green', linestyle='--', label='Mean')
    
    ax.text(x=pd.Timestamp('2024-08-15'), y=difference_mean + std_error, s=f'Mean: {difference_mean:.2f}', color='green')
    
    ax.axhline(y=difference_mean + 1.96 * difference_std, color='red', linestyle='--', label='1.96 * Std')
    # add text over the horizontal line
    ax.text(x=pd.Timestamp('2024-08-15'), y=difference_mean + 1.96 * difference_std + std_error, s=f'+ 1.96 * Std', color='red')
    
    ax.axhline(y=difference_mean - 1.96 * difference_std, color='red', linestyle='--', label='-1.96 * Std')
    # add text over the horizontal line
    ax.text(x=pd.Timestamp('2024-08-15'), y=difference_mean - 1.96 * difference_std + std_error, s=f'-1.96 * Std', color='red')
    
    ax.axhline(y=0, color='black', linestyle='--')


    plt.annotate(
        f'Std error: {std_error:.2f}\nDifference mean CI: {difference_mean:.2f} ± {ci_difference_mean:.2f}',
        xy=(0.05, 0.9),
        xycoords='axes fraction',
        bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="lightblue"),
        color='green'
    )
    
    plt.title(f'{feature} - Bland-Altman Plot')
    plt.tight_layout()

    # display Bland-Altman plot
    if not os.path.exists(os.path.join(plot_folder, "Bland-Altman", "Supply Points", "All_Date")):
        os.makedirs(os.path.join(plot_folder, "Bland-Altman", "Supply Points", "All_Date"))
        
    feature_ = feature.replace("/", "_")

    plt.savefig(os.path.join(plot_folder, "Bland-Altman", "Supply Points", "All_Date", f'{feature_}.png'), dpi=300)
    plt.close(f)

    # plt.show()

In [ ]:
%%script false --no-raise-error
# For each supply point

for code in grab_df["Code"].unique():
    for feature in feature_mapping.values():
        g_df = grab_df[grab_df["Code"] == code].copy()
        
        # if code == "Berna" and feature == "Free Chlorine (mg/L)":
        #     pass

        s_df = sensor_dict[code].copy()

        # moving average on sensor data
        s_df["DateTime"] = pd.to_datetime(s_df["DateTime"])
        s_df.set_index("DateTime", inplace=True)
        # 2 hours moving average
        s_df = s_df.rolling(window=4 * 2).mean()
        
        # fix the date of the sensor data to have a frequency of 15 minutes for easier comparison and interpolate to not have nan value
        s_df = s_df.resample("15min").mean().interpolate(method="time")

        sensor_start_date = s_df.index.dropna().min().strftime("%Y-%m-%d")
        sensor_end_date = s_df.dropna().index.max().strftime("%Y-%m-%d")
        
        g_df.set_index("DateTime", inplace=True)
        
        g_df = g_df[(g_df.index >= sensor_start_date) & (g_df.index <= sensor_end_date)]
        
        g_df = g_df[feature]
        g_df.dropna(inplace=True)
        
        # keep only the sensor values that have the date in the grab data and the hour is between 9 and 11
        
        dates = pd.Series(s_df.index.date, index=s_df.index).isin(g_df.index.date)
        dates = dates[dates.values]
        
        s_df = s_df.loc[dates.index]
        s_df = s_df[(s_df.index.hour == 10) & (s_df.index.minute >= 0) & (s_df.index.minute <= 14)]
        
        # if there is more than one value for the same date, take the mean
        s_df = s_df.groupby(s_df.index.date).mean()

        
        f, ax = plt.figure(figsize=(10, 6)), plt.gca()
        
        # try:
        #     sm.graphics.mean_diff_plot(g_df.values, s_df[feature].values, ax = ax)
        # except:
        #     pass
        sm.graphics.mean_diff_plot(g_df.values, s_df[feature].values, ax = ax)
        
        
        plt.title(f'{code} - {feature}')
        
        difference_mean = np.mean(g_df.values - s_df[feature].values)
        difference_std = np.std(g_df.values - s_df[feature].values)
        std_error = difference_std / np.sqrt(g_df.shape[0])
        
        ci_difference_mean = 1.96 * std_error
        
        
        plt.annotate(
            f'Std error: {std_error:.2f}\nDifference mean CI: {difference_mean:.2f} ± {ci_difference_mean:.2f}',
            xy=(0.05, 0.9),
            xycoords='axes fraction',
            bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="lightblue"),
            color='red'
        )
        
        

        #display Bland-Altman plot
        if not os.path.exists(os.path.join(plot_folder, "Bland-Altman", "Supply Points", code)):
            os.makedirs(os.path.join(plot_folder, "Bland-Altman", "Supply Points", code))
            
        feature_ = feature.replace("/", "_")
        
        plt.savefig(os.path.join(plot_folder, "Bland-Altman", "Supply Points", code, f'{feature_}.png'), dpi=300)
        plt.close(f)
        
        # plt.show()

# Check Measurement Time between Supply Points

In [ ]:
# check if the timestamps are the same for each code

common_dates = pd.Series()

for code in grab_df["Code"].unique():
    
    code_df = grab_df[grab_df["Code"] == code]["DateTime"].copy()
        
        
    if common_dates.empty:
        common_dates = code_df
    
    else:
        common_dates = pd.Series(list(set(common_dates).intersection(set(code_df))))
        
common_dates = common_dates.sort_values()

common_dates

In [ ]:
# cannot compare measurements on the same day, because the grab samples are taken at different times

In [ ]:
for feature in list(feature_mapping.values()):
    fig = go.Figure()
    
    for code in grab_df["Code"].unique():
        code_df = grab_df[grab_df["Code"] == code].copy()
        
        fig.add_trace(
            go.Scatter(
                x=code_df["DateTime"], y=code_df[feature], mode="markers", name=code
            )
        )
        
    fig.update_layout(
        title=feature,
        xaxis_title="DateTime",
        yaxis_title=feature
    )
    
    fig.show()

# Data Imputation with MICE

In [ ]:
grab_df.columns.to_list()

In [ ]:
def replace_non_normal(row, column):
    return np.nan if row[column + "_label"] != "Normal" else row[column]

In [ ]:
# get the columns that contain the word 'label'
label_columns = [col for col in grab_df.columns if 'label' in col]

In [ ]:
label_removed_grab_df = grab_df.copy()

for column in grab_df.columns.difference(["Code", "DateTime"] + label_columns):
    label_removed_grab_df[column] = label_removed_grab_df.apply(
        lambda row: replace_non_normal(row, column), axis=1
    )

In [ ]:
# assert that the columns have been replaced correctly
for column in label_removed_grab_df.columns.difference(["Code", "DateTime"] + label_columns):
    # assert that all the values that are nan should have a label different from 'Normal'
    assert label_removed_grab_df[(label_removed_grab_df[column].isna()) & (label_removed_grab_df[column + "_label"] == 'Normal')].shape[0] == 0 

    # assert that all the values that are not nan should have a label equal to 'Normal'
    assert label_removed_grab_df[(label_removed_grab_df[column].notna()) & (label_removed_grab_df[column + "_label"] != 'Normal')].shape[0] == 0

In [ ]:
# for every feature, save the indexes of the values that are less than the LOQ
indexes_df = pd.DataFrame(columns=feature_mapping.values(), index=grab_df["Code"].unique())

for code in grab_df["Code"].unique():
    for feature in feature_mapping.values():
        df = grab_df[grab_df["Code"] == code].copy()
        
        df = df[df[feature + "_label"] == "Less than"]
        
        indexes_df.loc[code, feature] = df.index.to_list()
    
indexes_df

In [ ]:
# remove the label columns
label_removed_grab_df = label_removed_grab_df.drop(columns=label_columns)

In [ ]:
import miceforest as mf

In [ ]:
df = label_removed_grab_df.copy()

df = df[list(feature_mapping.values()) + ['DateTime'] + ['Code']]

# convert datetime column to float
df['DateTime'] = pd.to_numeric(df['DateTime'])
df['Code'] = df['Code'].astype('category')

df = df.reset_index(drop=True)

kernel = mf.ImputationKernel(
    data=df,
    variable_schema=df.columns.difference(['DateTime', 'Code']).to_list(),
    random_state=42,
    mean_match_strategy='shap',
)

In [ ]:
print(df[df['Code'] == code].isnull().all(axis=1).sum())  # Rows with all NaN

In [ ]:
print(df[df['Code'] == code].isnull().all(axis=0).sum())  # Columns with all NaN

In [ ]:
# Perform MICE imputation
kernel.mice(4, verbose=True)

In [ ]:
completed_dataset = kernel.complete_data(dataset=0)

In [ ]:
completed_dataset

In [ ]:
%%script false --no-raise-error
# comparison of the less than LOQ values before and after imputation


for feature in ['Free Chlorine (mg/L)', 'Color (CU)', 'Turbidity (NTU)']:
    
    fig = go.Figure()
    
    for code in grab_df["Code"].unique():
    
        indexes = indexes_df.loc[code, feature]
        
        df = label_removed_grab_df.copy()
        
        df = df[(df["Code"] == code) & (df.index.isin(indexes))]
        
        df = df[[feature]]
        
        df['Imputed'] = completed_dataset.loc[df.index, feature]
        
        
        fig.add_trace(
            go.Scatter(
                x=df.index, y=df['Imputed'], mode='markers', name=code
            )
        )
    if feature == 'Free Chlorine (mg/L)':
        loq_value = 0.04
    elif feature == 'Color (CU)':
        loq_value = 1
    else:
        loq_value = 0.3
        
    fig.add_hline(y=loq_value, line_dash='dash', line_color='black', opacity=0.3)
        
    
    fig.update_layout(
        title=f'{feature} - LOQ: {loq_value}',
        xaxis_title='Index',
        yaxis_title=feature
    )
    
    if not os.path.exists(os.path.join(plot_folder, 'Imputation', 'LOQ')):
            os.makedirs(os.path.join(plot_folder, 'Imputation', 'LOQ'))

    feature_ = feature.replace('/', '_')

    fig.write_image(os.path.join(plot_folder, 'Imputation', 'LOQ', f'{feature_}.png'), height=600, width=1200, scale=3)

    
    

In [ ]:
# convert the datetime column back to datetime
completed_dataset['DateTime'] = pd.to_datetime(completed_dataset['DateTime'])

## Bland-Altman Imputed Data

In [ ]:
%%script false --no-raise-error
# With all the supply points together

total_g_df = pd.DataFrame(columns=["Code", "DateTime", "Feature", "Value"])
total_s_df = pd.DataFrame(columns=["Code", "DateTime", "Feature", "Value"])

for code in grab_df["Code"].unique():
    for feature in feature_mapping.values():
        g_df = completed_dataset[completed_dataset["Code"] == code].copy()
        
        # if code == "Berna" and feature == "Free Chlorine (mg/L)":
        #     pass

        s_df = sensor_dict[code].copy()

        # moving average on sensor data
        s_df["DateTime"] = pd.to_datetime(s_df["DateTime"])
        s_df.set_index("DateTime", inplace=True)
        # 2 hours moving average
        s_df = s_df.rolling(window=4 * 2).mean()
        
        # fix the date of the sensor data to have a frequency of 15 minutes for easier comparison and interpolate to not have nan value
        s_df = s_df.resample("15min").mean().interpolate(method="time")

        sensor_start_date = s_df.index.dropna().min().strftime("%Y-%m-%d")
        sensor_end_date = s_df.dropna().index.max().strftime("%Y-%m-%d")
        
        g_df.set_index("DateTime", inplace=True)
        
        g_df = g_df[(g_df.index >= sensor_start_date) & (g_df.index <= sensor_end_date)]
        
        g_df = g_df[feature]
        g_df.dropna(inplace=True)
        
        # keep only the sensor values that have the date in the grab data and the hour is between 9 and 11
        
        dates = pd.Series(s_df.index.date, index=s_df.index).isin(g_df.index.date)
        dates = dates[dates.values]
        
        s_df = s_df.loc[dates.index]
        s_df = s_df[(s_df.index.hour == 10) & (s_df.index.minute >= 0) & (s_df.index.minute <= 14)]
        
        # if there is more than one value for the same date, take the mean
        s_df = s_df.groupby(s_df.index.date).mean()
        
        total_g_df = pd.concat([total_g_df, pd.DataFrame({"Code": code, "DateTime": g_df.index, "Feature": feature, "Value": g_df.values})])
        total_s_df = pd.concat([total_s_df, pd.DataFrame({"Code": code, "DateTime": s_df.index, "Feature": feature, "Value": s_df[feature].values})])


for feature in feature_mapping.values():
    
    g_df = total_g_df[total_g_df["Feature"] == feature]
    s_df = total_s_df[total_s_df["Feature"] == feature]
    
    g_df["DateTime"] = pd.to_datetime(g_df["DateTime"])
    s_df["DateTime"] = pd.to_datetime(s_df["DateTime"])
    
        
    df = pd.merge(g_df, s_df, on=["Code", "DateTime"], suffixes=("_Grab", "_Sensor"))
    df["Difference"] = df["Value_Grab"] - df["Value_Sensor"]
    df['Mean'] = (df["Value_Grab"] + df["Value_Sensor"]) / 2
    

    difference_mean = np.mean(df["Difference"].values)
    difference_std = np.std(df["Difference"].values)
    std_error = difference_std / np.sqrt(g_df.shape[0])

    ci_difference_mean = 1.96 * std_error
    
    
    f, ax = plt.subplots(1, 1, figsize=(15, 10))
    sns.scatterplot(data=df, x="Mean", y="Difference", hue="Code", ax=ax, s=100)
    
    ax.axhline(y=difference_mean, color='green', linestyle='--', label='Mean')
    
    ax.text(x=df['Mean'].quantile(0.97), y=difference_mean + std_error, s=f'Mean: {difference_mean:.2f}', color='green')
    
    ax.axhline(y=difference_mean + 1.96 * difference_std, color='red', linestyle='--', label='1.96 * Std')
    # add text over the horizontal line
    ax.text(x=df['Mean'].quantile(0.97), y=difference_mean + 1.96 * difference_std + std_error, s=f'+ 1.96 * Std', color='red')
    
    ax.axhline(y=difference_mean - 1.96 * difference_std, color='red', linestyle='--', label='-1.96 * Std')
    # add text over the horizontal line
    ax.text(x=df['Mean'].quantile(0.97), y=difference_mean - 1.96 * difference_std + std_error, s=f'-1.96 * Std', color='red')
    
    ax.axhline(y=0, color='black', linestyle='--')


    plt.annotate(
        f'Std error: {std_error:.2f}\nDifference mean CI: {difference_mean:.2f} ± {ci_difference_mean:.2f}',
        xy=(0.05, 0.9),
        xycoords='axes fraction',
        bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="lightblue"),
        color='green'
    )
    
    plt.title(f'{feature} - Bland-Altman Plot')
    plt.tight_layout()

    # display Bland-Altman plot
    if not os.path.exists(os.path.join(plot_folder, "Bland-Altman", "Supply Points", "Imputed")):
        os.makedirs(os.path.join(plot_folder, "Bland-Altman", "Supply Points", "Imputed"))
        
    feature_ = feature.replace("/", "_")

    plt.savefig(os.path.join(plot_folder, "Bland-Altman", "Supply Points", "Imputed", f'{feature_}.png'), dpi=300)
    plt.close(f)

    # plt.show()

# Data Imputation with LOQ/2

Value points with the label 'Less than' are imputed with LOQ/2, while value points with label 'NaN' are imputed with MICE

In [ ]:
def replace_loq(row, column):
    return row[column] if row[column + "_label"] != "Less than" else row[column] / 2

In [ ]:
label_columns = [col for col in grab_df.columns if 'label' in col]

In [ ]:
df = grab_df.copy()

for column in grab_df.columns.difference(["Code", "DateTime"] + label_columns):
    df[column] = df.apply(
        lambda row: replace_loq(row, column), axis=1
    )

In [ ]:
df = df.drop(columns=label_columns)

In [ ]:
df = df[list(feature_mapping.values()) + ['DateTime'] + ['Code']]

# convert datetime column to float
df['DateTime'] = pd.to_numeric(df['DateTime'])
df['Code'] = df['Code'].astype('category')

df = df.reset_index(drop=True)

In [ ]:
for code in grab_df["Code"].unique():
    if df[df['Code'] == code].isnull().all(axis=1).sum() > 0: # Rows with all NaN
        print(f'{code} has {df[df['Code'] == code].isnull().all(axis=1).sum()} rows with all NaN')

In [ ]:
for code in grab_df["Code"].unique():
    if df[df['Code'] == code].isnull().all(axis=0).sum() > 0: # Columns with all NaN
        print(f'{code} has {df[df["Code"] == code].isnull().all(axis=0).sum()} columns with all NaN')

In [ ]:
df

In [ ]:
# Perform MICE imputation

kernel = mf.ImputationKernel(
    data=df,
    variable_schema=df.columns.difference(['DateTime', 'Code']).to_list(),
    random_state=42,
    mean_match_strategy='shap',
)

kernel.mice(4, verbose=True)

In [ ]:
completed_dataset = kernel.complete_data(dataset=0)

In [ ]:
completed_dataset.head(5)

# UMAP Visualization

In [ ]:
house_codes = grab_df["Code"].unique()

code_mapping = {code: i for i, code in enumerate(house_codes)}
df = completed_dataset[['Code'] + list(feature_mapping.values())].copy()

df['Code'] = df['Code'].map(code_mapping)
df['Code'] = df['Code'].astype('category')

In [ ]:
# revert the code mapping
# df['Code'] = df['Code'].map({v: k for k, v in code_mapping.items()})

In [ ]:
df.shape

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
mapper = umap.UMAP(n_neighbors=10, random_state=42).fit(df[['Code'] + list(feature_mapping.values())])

In [ ]:
code_mapping

In [ ]:
umap.plot.points(mapper, labels=df['Code'])

In [ ]:
hover_data = pd.DataFrame({
    'index': np.arange(df.shape[0]),
    'label': df['Code']
})

hover_data['item'] = hover_data.label.map(
    {v: k for k, v in code_mapping.items()}
)

In [ ]:
p = umap.plot.interactive(mapper, labels=df['Code'], hover_data=hover_data, point_size=4)

In [ ]:
from bokeh.plotting import save, output_file

In [ ]:
output_file(os.path.join(plot_folder, 'Imputation', 'UMAP.html'))

save(p)

In [ ]:
# sembra che Tognazzi sia abbastanza distante dagli altri, anche il pair plot fa notare una leggera differenza rispetto agli altri

# Clustering

## COP - KMeans

### Trials

In [ ]:
from copkmeans.cop_kmeans import cop_kmeans

In [ ]:
df

In [ ]:
from itertools import combinations

In [ ]:
n_clusters = 5

must_link = []

code_column = df['Code']

for code in df['Code'].unique():
    # get all the pair combinations of the entries with same code to add to the must_link
    index_pairs = list(combinations(df[df['Code'] == code].index, 2)) 
    must_link.extend(index_pairs)

np_df = df[list(feature_mapping.values())].to_numpy()

clusters, centers = cop_kmeans(np_df, n_clusters, ml=must_link)

In [ ]:
df['Cluster'] = clusters
df['Code'] = code_column

In [ ]:
code_mapping

In [ ]:
centers

In [ ]:
for cluster in df['Cluster'].unique():
    print(f'Cluster {cluster}')
    codes = df[df['Cluster'] == cluster]['Code'].unique().tolist()
    # get the key from the value
    codes = [k for k, v in code_mapping.items() if v in codes]
    print(codes)

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy import sparse

In [ ]:
df.drop(columns=['Cluster'], inplace=True)

In [ ]:
# define the connectivity matrix such that each sample with the same code is connected
connectivity = np.zeros((df.shape[0], df.shape[0]))

for code in df['Code'].unique():
    # indexes = df[df['Code'] == code].index.to_numpy()
    # # Set connectivity for all pairs of samples with the same 'Code' to 1
    # connectivity[indexes[:, None], indexes] = 1
    
    index_pairs = list(combinations(df[df['Code'] == code].index, 2))
    for i, j in index_pairs:
        connectivity[i, j] = 1
        connectivity[j, i] = 1

connectivity = sparse.csr_matrix(connectivity)

In [ ]:
ward = AgglomerativeClustering(n_clusters=9, linkage='ward', connectivity=connectivity)

In [ ]:
ward.fit(df[['Code'] + list(feature_mapping.values())])

In [ ]:
df['Cluster'] = ward.labels_

In [ ]:
for cluster in df['Cluster'].unique():
    print(f'Cluster {cluster}')
    codes = df[df['Cluster'] == cluster]['Code'].unique().tolist()
    # get the key from the value
    codes = [k for k, v in code_mapping.items() if v in codes]
    print(codes)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# import cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
%%script false --no-raise-error
df1 = completed_dataset[completed_dataset['Code'] == 'Berna']['Temperature (°C)'].copy()
df2 = completed_dataset[completed_dataset['Code'] == 'Tabacchi']['Temperature (°C)'].copy()

print("Wasserstein no standardization:", wasserstein_distance(df1, df2))
scaler = MinMaxScaler()

df1 = pd.Series(scaler.fit_transform(df1.values.reshape(-1, 1)).flatten())

df2 = pd.Series(scaler.fit_transform(df2.values.reshape(-1, 1)).flatten())

print("Wasserstein with standardization:", wasserstein_distance(df1, df2))

In [ ]:
from itertools import combinations

In [ ]:
for house1, house2 in combinations(house_codes, 2):
    df1 = completed_dataset[completed_dataset['Code'] == house1][feature_mapping.values()].copy()
    df2 = completed_dataset[completed_dataset['Code'] == house2][feature_mapping.values()].copy()
    
    print(f"House1: {house1} - House2: {house2}")
    print(np.mean(cosine_similarity(df1, df2)))
    print()

In [ ]:
# standadization affects the wasserstein distance

### On THMs

#### Sum

In [ ]:
thms_columns = ['Bromodichloromethane (µg/L)', 'Bromoform (µg/L)', 'Chloroform (µg/L)', 'Dibromochloromethane (µg/L)']

In [ ]:
grab_df[['Code'] + thms_columns]

In [ ]:
house_codes = grab_df["Code"].unique()

code_mapping = {code: i for i, code in enumerate(house_codes)}

In [ ]:
df = grab_df[['Code'] + thms_columns].copy()

df['Code'] = df['Code'].map(code_mapping)
df['Code'] = df['Code'].astype('category')

In [ ]:
dataframe = df.copy()
dataframe = dataframe.dropna()

In [ ]:
dataframe['TTHM'] = dataframe[thms_columns].sum(axis=1)

In [ ]:
dataframe.reset_index(drop=True, inplace=True)

In [ ]:
must_link = []


for code in dataframe['Code'].unique():
    # get all the pair combinations of the entries with same code to add to the must_link
    index_pairs = list(combinations(dataframe[dataframe['Code'] == code].index, 2)) 
    must_link.extend(index_pairs)

np_df = dataframe['TTHM'].to_numpy().reshape(-1, 1)

In [ ]:
variances = []

for n_cluster in range(2, 10):
    clusters, centers = cop_kmeans(np_df, n_cluster, ml=must_link)
    
    dataframe['Cluster'] = clusters
    
    # compute the variance of each cluster
    variance = 0
    for cluster in dataframe['Cluster'].unique():
        cluster_df = dataframe[dataframe['Cluster'] == cluster].copy()
        variance += np.var(cluster_df['TTHM'])
    
    # compute average variance for n_cluster
    variance /= n_cluster    
    
    variances.append(variance)
    
    dataframe.drop(columns=['Cluster'], inplace=True)
    

In [ ]:
# plot the elbow curve
plt.plot(range(2, 10), variances)

plt.xlabel('Number of clusters')
plt.ylabel('Average Variance')
plt.show()




In [ ]:
# the optimal number of clusters is 3

In [ ]:
n_clusters = 3

must_link = []


for code in dataframe['Code'].unique():
    # get all the pair combinations of the entries with same code to add to the must_link
    index_pairs = list(combinations(dataframe[dataframe['Code'] == code].index, 2)) 
    must_link.extend(index_pairs)

np_df = dataframe['TTHM'].to_numpy().reshape(-1, 1)

clusters, centers = cop_kmeans(np_df, n_clusters, ml=must_link)

In [ ]:
dataframe['Cluster'] = clusters

In [ ]:
code_mapping

In [ ]:
centers

In [ ]:
for cluster in dataframe['Cluster'].unique():
    print(f'Cluster {cluster}')
    codes = dataframe[dataframe['Cluster'] == cluster]['Code'].unique().tolist()
    # get the key from the value
    codes = [k for k, v in code_mapping.items() if v in codes]
    print(codes)

In [ ]:
# plot the clusters
fig = go.Figure()

# for each code, use a different marker
# for each cluster, use a different color


symbols = ["circle", "square", "diamond", "cross", "triangle-up", "triangle-down", 
           "pentagon", "hexagon", "star"]

colors = ["red", "blue", "green", "purple",]

for i, cluster in enumerate(dataframe['Cluster'].unique()):
    for j, code in enumerate(dataframe['Code'].unique()):
        subset = dataframe[(dataframe["Cluster"] == cluster) & (dataframe["Code"] == code)]
        
        code_name = [k for k, v in code_mapping.items() if v == code][0]
        
        if not subset.empty:
            fig.add_trace(go.Scatter(
                x=subset['Code'].map({v: k for k, v in code_mapping.items()}),  # X-axis as the index
                y=subset["TTHM"],  # Y-axis as the variable
                mode="markers",
                marker=dict(
                    symbol=symbols[j],  # Unique symbol for each code
                    size=10,
                    line=dict(width=1),
                    color=colors[i],  # Unique color for each cluster
                ),
                name=f"{code_name}, Cluster: {cluster}"
            ))

fig.update_layout(
    title="Clusters",
    xaxis_title="Code",
    yaxis_title="TTHM"
)

fig.show()

#### Uniques

In [ ]:
thms_columns = ['Bromodichloromethane (µg/L)', 'Bromoform (µg/L)', 'Chloroform (µg/L)', 'Dibromochloromethane (µg/L)']

In [ ]:
grab_df[['Code'] + thms_columns]

In [ ]:
house_codes = grab_df["Code"].unique()

code_mapping = {code: i for i, code in enumerate(house_codes)}

In [ ]:
df = grab_df[['Code'] + thms_columns].copy()

df['Code'] = df['Code'].map(code_mapping)
df['Code'] = df['Code'].astype('category')

In [ ]:
dataframe = df.copy()
dataframe = dataframe.dropna()

In [ ]:
dataframe['TTHM'] = dataframe[thms_columns].sum(axis=1)

In [ ]:
dataframe.reset_index(drop=True, inplace=True)

In [ ]:
must_link = []


for code in dataframe['Code'].unique():
    # get all the pair combinations of the entries with same code to add to the must_link
    index_pairs = list(combinations(dataframe[dataframe['Code'] == code].index, 2)) 
    must_link.extend(index_pairs)

np_df = dataframe[thms_columns + ['TTHM']].to_numpy()

In [ ]:
variances = []

for n_cluster in range(2, 10):
    clusters, centers = cop_kmeans(np_df, n_cluster, ml=must_link)
    
    dataframe['Cluster'] = clusters
    
    # compute the variance of each cluster
    variance = 0
    for cluster in dataframe['Cluster'].unique():
        cluster_df = dataframe[dataframe['Cluster'] == cluster].copy()
        variance += np.var(cluster_df[thms_columns + ['TTHM']]).mean()
    
    # compute average variance for n_cluster
    variance /= n_cluster    
    
    variances.append(variance)
    
    dataframe.drop(columns=['Cluster'], inplace=True)
    

In [ ]:
# plot the elbow curve
plt.plot(range(2, 10), variances)

plt.xlabel('Number of clusters')
plt.ylabel('Average Variance')
plt.show()




In [ ]:
# the optimal number of clusters is 4

In [ ]:
n_clusters = 4

must_link = []


for code in dataframe['Code'].unique():
    # get all the pair combinations of the entries with same code to add to the must_link
    index_pairs = list(combinations(dataframe[dataframe['Code'] == code].index, 2)) 
    must_link.extend(index_pairs)

np_df = dataframe[thms_columns + ['TTHM']].to_numpy()

clusters, centers = cop_kmeans(np_df, n_clusters, ml=must_link)

In [ ]:
dataframe['Cluster'] = clusters

In [ ]:
code_mapping

In [ ]:
centers

In [ ]:
for cluster in dataframe['Cluster'].unique():
    print(f'Cluster {cluster}')
    codes = dataframe[dataframe['Cluster'] == cluster]['Code'].unique().tolist()
    # get the key from the value
    codes = [k for k, v in code_mapping.items() if v in codes]
    print(codes)

### On inputs only

In [ ]:
completed_dataset['DateTime'] = pd.to_datetime(completed_dataset['DateTime'])

In [ ]:
input_columns = list(feature_mapping.values())

In [ ]:
house_codes = completed_dataset["Code"].unique()

code_mapping = {code: i for i, code in enumerate(house_codes)}

In [ ]:
df = completed_dataset.copy()

df['Code'] = df['Code'].map(code_mapping)
df['Code'] = df['Code'].astype('category')

In [ ]:
dataframe = df.copy()
dataframe = dataframe.dropna()

In [ ]:
dataframe.reset_index(drop=True, inplace=True)

In [ ]:
scaler = MinMaxScaler()

dataframe[input_columns] = pd.DataFrame(scaler.fit_transform(dataframe[input_columns]), columns=input_columns)

In [ ]:
must_link = []


for code in dataframe['Code'].unique():
    # get all the pair combinations of the entries with same code to add to the must_link
    index_pairs = list(combinations(dataframe[dataframe['Code'] == code].index, 2)) 
    must_link.extend(index_pairs)

np_df = dataframe[input_columns].to_numpy()

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

In [ ]:
random.seed(seed)
np.random.seed(seed)

variances = []
sil_scores = []
db_scores = []
ch_scores = []


total_points = dataframe.shape[0]

for n_cluster in range(2, 10):
    clusters, centers = cop_kmeans(np_df, n_cluster, ml=must_link)
    
    sil_score = silhouette_score(np_df, clusters)
    db_score = davies_bouldin_score(np_df, clusters)
    ch_score = calinski_harabasz_score(np_df, clusters)
    
    dataframe['Cluster'] = clusters
    
    # compute the variance of each cluster
    variance = 0
    for cluster in dataframe['Cluster'].unique():
        cluster_df = dataframe[dataframe['Cluster'] == cluster].copy()
        variance += np.var(cluster_df[input_columns], axis=0).mean() * cluster_df.shape[0] / total_points
    
    # compute average variance for n_cluster
    variance /= n_cluster    
    
    variances.append(variance)
    sil_scores.append(sil_score)
    db_scores.append(db_score)
    ch_scores.append(ch_score)
    
    dataframe.drop(columns=['Cluster'], inplace=True)
    

In [ ]:
# normalize the scores
variances = np.array(variances)
sil_scores = np.array(sil_scores)
db_scores = np.array(db_scores)
ch_scores = np.array(ch_scores)

variances = scaler.fit_transform(variances.reshape(-1, 1)).flatten()
sil_scores = scaler.fit_transform(sil_scores.reshape(-1, 1)).flatten()
db_scores = scaler.fit_transform(db_scores.reshape(-1, 1)).flatten()
ch_scores = scaler.fit_transform(ch_scores.reshape(-1, 1)).flatten()


plt.plot(range(2, 10), variances, label='Weighted Average Variance')
plt.plot(range(2, 10), sil_scores, label='Silhouette Score')
plt.plot(range(2, 10), db_scores, label='Davies Bouldin Score')
plt.plot(range(2, 10), ch_scores, label='Calinski Harabasz Score')


plt.xlabel('Number of clusters')
plt.ylabel('(Normalized) Scores')

plt.legend()

plt.show()




In [ ]:
# a good number of clusters is 3

In [ ]:
n_clusters = 3

must_link = []


for code in dataframe['Code'].unique():
    # get all the pair combinations of the entries with same code to add to the must_link
    index_pairs = list(combinations(dataframe[dataframe['Code'] == code].index, 2)) 
    must_link.extend(index_pairs)

np_df = dataframe[input_columns].to_numpy()

clusters, centers = cop_kmeans(np_df, n_clusters, ml=must_link, max_iter=1000)

In [ ]:
dataframe['Cluster'] = clusters

In [ ]:
code_mapping

In [ ]:
for cluster in dataframe['Cluster'].unique():
    print(f'Cluster {cluster}')
    codes = dataframe[dataframe['Cluster'] == cluster]['Code'].unique().tolist()
    # get the key from the value
    codes = [k for k, v in code_mapping.items() if v in codes]
    print(codes)

### On joint with only TTHMs

In [ ]:
grab_df

In [ ]:
completed_dataset

In [ ]:
df = completed_dataset.copy()

In [ ]:
thms_columns =list(targets_mapping.values())


In [ ]:
tthms_df = grab_df[thms_columns].copy()
tthms_df.reset_index(drop=True, inplace=True)

In [ ]:
# append the thms columns
df['TTHMs'] = tthms_df.sum(axis=1, min_count=len(thms_columns)).round(2)

In [ ]:
df

In [ ]:
joint_columns = list(feature_mapping.values()) + ['TTHMs']

In [ ]:
house_codes = df["Code"].unique()

code_mapping = {code: i for i, code in enumerate(house_codes)}

In [ ]:
df['Code'] = df['Code'].map(code_mapping)
df['Code'] = df['Code'].astype('category')

In [ ]:
dataframe = df.copy()
dataframe = dataframe.dropna()

In [ ]:
dataframe.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

dataframe[joint_columns] = pd.DataFrame(scaler.fit_transform(dataframe[joint_columns]), columns=joint_columns)

In [ ]:
from itertools import combinations

In [ ]:
must_link = []


for code in dataframe['Code'].unique():
    # get all the pair combinations of the entries with same code to add to the must_link
    index_pairs = list(combinations(dataframe[dataframe['Code'] == code].index, 2)) 
    must_link.extend(index_pairs)

np_df = dataframe[joint_columns].to_numpy()

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

In [ ]:
random.seed(1)
np.random.seed(seed=1)

variances = []
sil_scores = []
db_scores = []
ch_scores = []

total_points = dataframe.shape[0]

variance = np.var(dataframe[joint_columns], axis=0).mean()
variances.append(variance)

for n_cluster in range(2, 10):
    clusters, centers = cop_kmeans(np_df, n_cluster, ml=must_link)
    
    sil_score = silhouette_score(np_df, clusters)
    db_score = davies_bouldin_score(np_df, clusters)
    ch_score = calinski_harabasz_score(np_df, clusters)
    
    dataframe['Cluster'] = clusters
    
    # compute the variance of each cluster
    variance = 0
    for cluster in dataframe['Cluster'].unique():
        cluster_df = dataframe[dataframe['Cluster'] == cluster].copy()
        variance += np.var(cluster_df[joint_columns], axis=0).mean() * cluster_df.shape[0] / total_points
    
    # compute average variance for n_cluster
    variance /= n_cluster    
    
    variances.append(variance)
    sil_scores.append(sil_score)
    db_scores.append(db_score)
    ch_scores.append(ch_score)
    
    dataframe.drop(columns=['Cluster'], inplace=True)
    

In [ ]:
# normalize the scores
variances = np.array(variances)
sil_scores = np.array(sil_scores)
db_scores = np.array(db_scores)
ch_scores = np.array(ch_scores)

variances = scaler.fit_transform(variances.reshape(-1, 1)).flatten()
sil_scores = scaler.fit_transform(sil_scores.reshape(-1, 1)).flatten()
db_scores = scaler.fit_transform(db_scores.reshape(-1, 1)).flatten()
ch_scores = scaler.fit_transform(ch_scores.reshape(-1, 1)).flatten()


plt.plot(range(1, 10), variances, label='Weighted Average Variance')
plt.plot(range(2, 10), sil_scores, label='Silhouette Score')
plt.plot(range(2, 10), db_scores, label='Davies Bouldin Score')
plt.plot(range(2, 10), ch_scores, label='Calinski Harabasz Score')

plt.xlabel('Number of clusters')
plt.ylabel('(Normalized) Scores')

plt.legend()

plt.show()

In [ ]:
# a good number of clusters is 3

In [ ]:
random.seed(seed)
np.random.seed(seed=seed)
n_clusters = 3

must_link = []


for code in dataframe['Code'].unique():
    # get all the pair combinations of the entries with same code to add to the must_link
    index_pairs = list(combinations(dataframe[dataframe['Code'] == code].index, 2)) 
    must_link.extend(index_pairs)

np_df = dataframe[joint_columns].to_numpy()

clusters, centers = cop_kmeans(np_df, n_clusters, ml=must_link, max_iter=1000)

In [ ]:
dataframe['Cluster'] = clusters

In [ ]:
code_mapping

In [ ]:
for cluster in dataframe['Cluster'].unique():
    print(f'Cluster {cluster}')
    codes = dataframe[dataframe['Cluster'] == cluster]['Code'].unique().tolist()
    # get the key from the value
    codes = [k for k, v in code_mapping.items() if v in codes]
    print(codes)

In [ ]:
dataframe

In [ ]:
# remap the code to the original code
dataframe['Code'] = dataframe['Code'].map({v: k for k, v in code_mapping.items()})

In [ ]:
dataframe

In [ ]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df['Code'] = dataframe['Code']
df['Cluster'] = dataframe['Cluster']

In [ ]:
df

In [ ]:
df.to_excel(os.path.join(clean_data_folder, 'modelling_grab.xlsx'), index=False)